<a href="https://colab.research.google.com/github/HodaMemar/Patient-Similarity-through-Representation/blob/main/Graph_Sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import networkx as nx
from nltk.tokenize import word_tokenize

Stage refers to the set of quadraples that are used to make the graph.
This collection is arranged according to the time dimension

In [ ]:

Stage_df1=pd.read_csv('...\\Stage.csv')

We designed two types of stages to design two methods U-TTree and H_U_TTree. which is in H_U_TTree data extracted from past medical history.

In [ ]:
list_Subject_id=Stage_df1['Subject_id'].unique().tolist()
list_Events=['procedure','Drug','past medical history']
foot_print=[]
Subgraph_id_num=3
list_sen=[]
counter=1
print(len(list_Subject_id))
for row_1 in list_Subject_id:
    root=''
    G = nx.Graph()
    foot_print=[]
    foot_print.append('DiseaseDisorderMention')
    foot_print.append('procedure')
    foot_print.append('Drug')
    label="Patient_"
    foot_print.append(label+str(row_1))
    G.add_node(label+str(row_1))
    root=label+str(row_1)
    for row_2 in range(Subgraph_id_num):
            label="SG_"
            
            G.add_node(label+str(row_2))
            foot_print.append(label+str(row_2))
            G.add_edge("Patient_"+str(row_1), label+str(row_2))
            if row_2==1:
                list_Timestame=Stage_df1.loc[Stage_df1['Subject_id']==row_1]['Timestame_id'].unique().tolist()
                list_Timestame.sort()
                for index, row_3 in enumerate(list_Timestame):
                    label="T_"
                    
                    current=label+str(index)

                    G.add_node(current)
                    foot_print.append(current)
                    
                    if index==0:
                            previous="SG_"+str(row_2)
                            
                            
                    elif index>0:
                            previous = label+str(index-1)
                            
                    G.add_edge(previous, current)
                    foot_print.append(previous)
                    
                    list_Events=[]        
                    list_Events=list(Stage_df1.loc[(Stage_df1['Subject_id']==row_1) & (Stage_df1['Subgraph_id']==row_2) & (Stage_df1['Timestame_id']==row_3)]['Event type'].unique())
                        
                    #print(list_Events)
                    for row_4 in list_Events:
                            label="T"+str(index) + "_E "
                            
                            foot_print.append("T"+str(index) + "_E")
                           
                            inx=list(Stage_df1.index[(Stage_df1['Subject_id']==row_1) & (Stage_df1['Subgraph_id']==row_2) & (Stage_df1['Timestame_id']==row_3)& (Stage_df1['Event type']==row_4)])
                            
                            entity=label +str(row_4)+' '+str(Stage_df1.iloc[inx[0]]['entity'])
                            value=''
                            for index, row_5 in enumerate(inx):
                                value=value+' '+Stage_df1.iloc[row_5]['value']
                            value=label +str(row_4)+' '+value

                            G.add_node(entity+' '+value)
                            G.add_edge(current, entity+' '+value)
    
    #graph_plit(G)
    list_sen.append([row_1,traverse_tree(G,root,foot_print)])
    print(counter,row_1)
    counter=counter+1    

In [ ]:
sen=pd.DataFrame(list_sen,columns=['Subject_id','String']) 

In [ ]:
def traverse_tree(G,source,foot_print):
    T = nx.bfs_tree(G, source=root)
    bfs_string=list(T.nodes())
    
    new=[]
    stopwords = foot_print
    tokenized_sents = [word_tokenize(i) for i in bfs_string]
    for i in tokenized_sents:
        words=i
        for word in list(words):  # iterating on a copy since removing will mess things up
            if word in stopwords:
                words.remove(word)
        if(len(words)>0):
            new.append(words)  
    sen=''
    for i in range(len(new)):
        sen=sen +' '+(' '.join(map(str, new[i])))
    return(sen) 

In this cell we can plot a generated graph.

In [ ]:
def graph_plit(G):
    
    G.nodes
    plt.figure(1)
    plt.figure(2,figsize=(12,12))
    
    list_root=[label+str(row_1)]
    list_subgraph=['SG_1','SG_2','SG_0']
    list_Time=['T_1','T_2']
    pos=nx.spring_layout(G)

    nx.draw(G, pos=pos, node_size = 1200, with_labels=True,node_color = 'silver')
    nx.draw_networkx_nodes(G, pos=pos,nodelist = list_root, node_color = 'red', node_size = 1500)
    nx.draw_networkx_nodes(G, pos=pos,nodelist = list_subgraph, node_color = 'orange', node_size = 1400)
    nx.draw_networkx_nodes(G, pos=pos,nodelist = list_Time, node_color = 'gold', node_size = 1300)
    nx.draw(G,with_labels=True,node_size = 1500,node_color = 'cyan')
    nx.draw_networkx_nodes(G, nodelist = ['Patient_1'], node_color = 'red', node_size = 200)
    plt.show()

When method breadth first search is used to traverse the generated tree, the following alternative way can be used to generate compounds.
What is important is to maintain the order of clinical events in each time window. 

In [ ]:
ls_doc_patient=[]
for i in range(len(ls_patient)):
    try:
    #for each patients
        print(ls_patient[i])
        df_tmp=Stage_Total.loc[Stage_Total['admission_Id']==ls_patient[i]]
        ls_time=df_tmp.Timestame_id.unique().tolist()
        ls_time.sort()
        UnD=Stage_df_Note.loc[(Stage_df_Note['ID']==ls_patient[i]),['ICD'] ].values[0]
       

        doc_S_Non_TT=''
        doc_U_Non_TT=''
        doc_S_TT=''
        doc_S_U_TT=''
        doc_S_U_TT_R='' 
        for j in range(len(ls_time)):
            # in each time windows
            df_tmp_ent=df_tmp.loc[df_tmp['Timestame_id']==ls_time[j]]
            df_tmp_ent=df_tmp_ent.sort_values('entity')

            tmp_S_Non_TT=''
            tmp_U_Non_TT=''
            tmp_S_TT=''
            tmp_S_U_TT=''
            tmp_S_U_TT_R=''
            
            for k in range(len(df_tmp_ent)):
                tmp_S_Non_TT=tmp_S_Non_TT+ ' ' + re.sub(r"\W+|_-", "", df_tmp_ent.iloc[k].entity) 
            doc_S_Non_TT=doc_S_Non_TT+ tmp_S_Non_TT   
    #---------------------------------------------------------        
            for k in range(len(df_tmp_ent)):
                tmp_S_TT=tmp_S_TT + re.sub(r"\W+|_-", "", df_tmp_ent.iloc[k].entity) + \
                                               re.sub(r"\W+|_-", "", df_tmp_ent.iloc[k].value)
            doc_S_TT=doc_S_TT+ ' ' +tmp_S_TT    
    #---------------------------------------------------------                                                

            tmp_S_U_TT_R=tmp_S_U_TT_R+' '+tmp_S_TT+' '+ re.sub(r"\W+|_-", " ", str(UnD))
            doc_S_U_TT_R=doc_S_U_TT_R+ ' ' +tmp_S_U_TT_R

        doc_S_U_TT=doc_S_TT + ' ' + UnD
        doc_U_Non_TT=df_U_pref.loc[df_U['ID']==ls_patient[i]].Doc
        ls_doc_patient.append([ls_patient[i],doc_S_Non_TT,doc_U_Non_TT,doc_S_TT,doc_S_U_TT,doc_S_U_TT_R])
            
    except:       
        print('error')

        
        
        
st=pd.DataFrame(ls_doc_patient,columns=['ID','doc_S_Non_TT','doc_U_Non_TT','doc_S_TT','doc_S_U_TT','doc_S_U_TT_R']) 
st